<a href="https://colab.research.google.com/github/phanhuy0410/LuanVan/blob/main/Forensic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tarfile
from pathlib import Path

BASE_DIR = Path("/content/drive/MyDrive/DARPA_TC")
OUTPUT_DIR = BASE_DIR / "E3_extracted"
OUTPUT_DIR.mkdir(exist_ok=True)

tar_files = [
    "ta1-theia-e3-official-1r.json.tar.gz",
    "ta1-theia-e3-official-3.json.tar.gz",
    "ta1-theia-e3-official-5m.json.tar.gz",
    "ta1-theia-e3-official-6r.json.tar.gz"
]

for tar_name in tar_files:
    scenario = tar_name.split("-")[-1].replace(".json.tar.gz", "")
    extract_dir = OUTPUT_DIR / scenario
    extract_dir.mkdir(exist_ok=True)

    tar_path = BASE_DIR / tar_name
    print(f"Extracting {tar_name} → {extract_dir}")

    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=extract_dir)

print("✅ Done extracting all 4 files")



Extracting ta1-theia-e3-official-1r.json.tar.gz → /content/drive/MyDrive/DARPA_TC/E3_extracted/1r


/tmp/ipython-input-1998673097.py:24: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_dir)


Extracting ta1-theia-e3-official-3.json.tar.gz → /content/drive/MyDrive/DARPA_TC/E3_extracted/3
Extracting ta1-theia-e3-official-5m.json.tar.gz → /content/drive/MyDrive/DARPA_TC/E3_extracted/5m
Extracting ta1-theia-e3-official-6r.json.tar.gz → /content/drive/MyDrive/DARPA_TC/E3_extracted/6r
✅ Done extracting all 4 files


In [1]:
import json
from pathlib import Path
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq

DATA_DIR = Path("/content/drive/MyDrive/DARPA_TC/E3_extracted")
LABEL_FILE = Path("/content/drive/MyDrive/DARPA_TC/multiclass.json")
OUTPUT_PARQUET = "/content/drive/MyDrive/DARPA_TC/darpaTC_theia_labeled.parquet"

BATCH_SIZE = 100_000
with open(LABEL_FILE) as f:
    label_spec = json.load(f)

LABEL_MAPPING = {
    int(k): v["name"]
    for k, v in label_spec["label_definition"].items()
}

INDICATORS = label_spec["indicators"]
principal_info = {}

def parse_principal(record):
    datum = record.get("root", {}).get("datum", {})
    p = datum.get("com.bbn.tc.schema.avro.cdm18.Principal")
    if not p:
        return
    principal_info[p["uuid"]] = {
        "uid": p.get("uid"),
        "username": p.get("username")
    }
process_to_principal = {}

def parse_subject(record):
    datum = record.get("root", {}).get("datum", {})
    s = datum.get("com.bbn.tc.schema.avro.cdm18.Subject")
    if not s:
        return
    puuid = s.get("principal", {}).get("uuid")
    if puuid:
        process_to_principal[s["uuid"]] = puuid

def match_label(proc_name, obj_path, obj_ip):
    labels = []
    for ind in INDICATORS:
        if ind["type"] == "process_name" and proc_name == ind["value"]:
            labels.append(ind["label"])
        elif ind["type"] == "file_path" and obj_path == ind["value"]:
            labels.append(ind["label"])
        elif ind["type"] == "ip" and obj_ip == ind["value"]:
            labels.append(ind["label"])
    return max(labels) if labels else 0

def parse_event(record, scenario):
    datum = record.get("root", {}).get("datum", {})
    evt = datum.get("com.bbn.tc.schema.avro.cdm18.Event")
    if not evt:
        return None

    subj = evt.get("subject", {})
    obj = evt.get("object", {})

    proc_uuid = subj.get("uuid")
    proc_name = subj.get("name")

    obj_path = obj.get("path")
    obj_ip = obj.get("remoteAddress") or obj.get("ip")

    label_id = match_label(proc_name, obj_path, obj_ip)

    principal_uuid = process_to_principal.get(proc_uuid)
    user = principal_info.get(principal_uuid, {})

    return {
        "timestamp": evt.get("timestampNanos", 0) // 1_000_000_000,
        "operation": evt.get("predicate"),

        "proc_uuid": proc_uuid,
        "proc_name": proc_name,

        "user_uid": user.get("uid"),
        "username": user.get("username"),

        "obj_type": obj.get("type"),
        "obj_path": obj_path,
        "obj_ip": obj_ip,

        "label_id": label_id,
        "label_name": LABEL_MAPPING[label_id],
        "scenario": scenario
    }
writer = None
buffer = []

# ---------- PASS 1 & 2 ----------
print("🔍 Building principal & process maps...")
for scenario_dir in DATA_DIR.iterdir():
    if not scenario_dir.is_dir():
        continue
    for f in scenario_dir.glob("*.json*"):
        with open(f) as fh:
            for line in fh:
                rec = json.loads(line)
                parse_principal(rec)
                parse_subject(rec)

# ---------- PASS 3 ----------
print("🧠 Parsing events & labeling...")
for scenario_dir in DATA_DIR.iterdir():
    if not scenario_dir.is_dir():
        continue

    scenario = scenario_dir.name
    for f in scenario_dir.glob("*.json*"):
        with open(f, buffering=1024*1024) as fh:
            for line in tqdm(fh, desc=f.name):
                rec = json.loads(line)
                row = parse_event(rec, scenario)
                if not row:
                    continue

                buffer.append(row)
                if len(buffer) >= BATCH_SIZE:
                    table = pa.Table.from_pylist(buffer)
                    if writer is None:
                        writer = pq.ParquetWriter(
                            OUTPUT_PARQUET,
                            table.schema,
                            compression="snappy"
                        )
                    writer.write_table(table)
                    buffer.clear()

# ---------- FLUSH ----------
if buffer:
    writer.write_table(pa.Table.from_pylist(buffer))
if writer:
    writer.close()

print("✅ DONE:", OUTPUT_PARQUET)


🔍 Building principal & process maps...
🧠 Parsing events & labeling...


ta1-theia-e3-official-1r.json: 5000000it [01:48, 45872.67it/s]
ta1-theia-e3-official-1r.json.1: 5000000it [01:39, 50477.01it/s]
ta1-theia-e3-official-1r.json.2: 5000000it [01:36, 51988.39it/s]
ta1-theia-e3-official-1r.json.3: 5000000it [01:40, 49806.02it/s]
ta1-theia-e3-official-1r.json.4: 5000000it [02:10, 38292.20it/s]
ta1-theia-e3-official-1r.json.5: 5000000it [01:51, 44806.13it/s]
ta1-theia-e3-official-1r.json.6: 5000000it [01:42, 48700.33it/s]
ta1-theia-e3-official-1r.json.7: 5000000it [01:43, 48307.35it/s]
ta1-theia-e3-official-1r.json.8: 5000000it [01:42, 48827.34it/s]
ta1-theia-e3-official-1r.json.9: 1761949it [00:35, 49112.66it/s]
ta1-theia-e3-official-3.json: 1588762it [00:32, 48681.76it/s]
ta1-theia-e3-official-5m.json: 1090138it [00:23, 46310.84it/s]
ta1-theia-e3-official-6r.json: 5000000it [01:55, 43405.67it/s]
ta1-theia-e3-official-6r.json.1: 5000000it [01:42, 48970.54it/s]
ta1-theia-e3-official-6r.json.2: 5000000it [01:37, 51511.64it/s]
ta1-theia-e3-official-6r.json.3: 5

✅ DONE: /content/drive/MyDrive/DARPA_TC/darpaTC_theia_labeled.parquet


In [2]:
import pyarrow.parquet as pq
from collections import Counter

PARQUET_FILE = "/content/drive/MyDrive/DARPA_TC/darpaTC_theia_labeled.parquet"
BATCH_SIZE = 100_000

pf = pq.ParquetFile(PARQUET_FILE)

label_counter = Counter()
total_rows = 0

for batch in pf.iter_batches(batch_size=BATCH_SIZE, columns=["label_name"]):
    df = batch.to_pandas()
    label_counter.update(df["label_name"])
    total_rows += len(df)

print("📊 Tổng số event:", total_rows)
print("📊 Phân bố hành vi:")
for label, cnt in label_counter.items():
    print(f"{label:30s} {cnt}")


FileNotFoundError: [Errno 2] Failed to open local file '/content/drive/MyDrive/DARPA_TC/darpaTC_theia_labeled.parquet'. Detail: [errno 2] No such file or directory

In [ ]:
import json
from pathlib import Path
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import gc

# --- CẤU HÌNH ---
DATA_DIR = Path("/content/drive/MyDrive/DARPA_TC/E3_extracted")
# File JSON chứa danh sách nhãn (Ground Truth)
LABEL_FILE = Path("/content/drive/MyDrive/DARPA_TC/multiclass.json")
OUTPUT_PARQUET = "/content/drive/MyDrive/DARPA_TC/darpaTC_theia_labeled.parquet"
BATCH_SIZE = 100_000

# Load nhãn
with open(LABEL_FILE) as f:
    label_spec = json.load(f)

# Mapping ID -> Name (VD: 1 -> "Initial Access")
LABEL_MAPPING = label_spec["label_mapping"]
INDICATORS = label_spec["indicators"]

# --- GLOBAL MAPS (Lưu trên RAM để tra cứu nhanh) ---
principal_info = {}       # User UUID -> {uid, username}
subject_info = {}         # Subject UUID -> {principal_uuid, proc_name}
object_info = {}          # Object UUID -> {path, ip, type}

# Hàm trợ giúp: Lấy nội dung bên trong "datum" an toàn
def get_datum(record):
    # Dựa trên hình ảnh bạn gửi: root -> datum
    return record.get("root", {}).get("datum", {})

# ---------------------------------------------------------
# HÀM PASS 1: XÂY DỰNG TỪ ĐIỂN (MAPPING UUID -> INFO)
# ---------------------------------------------------------
def parse_metadata(record):
    datum = get_datum(record)
    if not datum: return

    # 1. Parse Principal (User)
    p = datum.get("com.bbn.tc.schema.avro.cdm18.Principal")
    if p:
        principal_info[p["uuid"]] = {
            "uid": p.get("userId"),
            "username": p.get("username")
        }
        return

    # 2. Parse Subject (Process) - Quan trọng để biết tên Process
    s = datum.get("com.bbn.tc.schema.avro.cdm18.Subject")
    if s:
        # Lấy tên process. Ưu tiên 'properties.map.path', fallback sang 'cmdLine'
        proc_name = "unknown"

        # Thử lấy path từ properties
        props = s.get("properties", {}).get("map", {})
        if props and "path" in props:
             proc_name = str(props["path"]).split('/')[-1]

        # Nếu không có path, thử cmdLine
        elif s.get("cmdLine"):
             # CmdLine ví dụ: "/bin/bash -c ..." -> lấy "bash"
             proc_name = str(s["cmdLine"]).split(' ')[0].split('/')[-1]

        subject_info[s["uuid"]] = {
            "principal_uuid": s.get("principal", {}).get("uuid") if isinstance(s.get("principal"), dict) else s.get("principal"),
            "proc_name": proc_name
        }
        return

    # 3. Parse FileObject - Quan trọng để biết file nào bị truy cập
    f_obj = datum.get("com.bbn.tc.schema.avro.cdm18.FileObject")
    if f_obj:
        # Lấy path từ properties
        props = f_obj.get("properties", {}).get("map", {})
        path = props.get("path")
        if path:
            object_info[f_obj["uuid"]] = {"path": path, "type": "FILE"}
        return

    # 4. Parse NetFlowObject - Quan trọng để biết IP kết nối
    n_obj = datum.get("com.bbn.tc.schema.avro.cdm18.NetFlowObject")
    if n_obj:
        ip = n_obj.get("remoteAddress")
        if ip:
            object_info[n_obj["uuid"]] = {"ip": ip, "type": "NETFLOW"}
        return

# ---------------------------------------------------------
# HÀM PASS 2: XỬ LÝ SỰ KIỆN (EVENT) & GÁN NHÃN
# ---------------------------------------------------------
def match_label(proc_name, obj_path, obj_ip):
    labels = []
    # Dùng key "default_label" khớp với file JSON ground truth
    target_key = "default_label"

    for ind in INDICATORS:
        # So sánh tên Process
        if ind["type"] == "process_name" and proc_name == ind["value"]:
            labels.append(ind[target_key])
        # So sánh đường dẫn File
        elif ind["type"] == "file_path" and obj_path == ind["value"]:
            labels.append(ind[target_key])
        # So sánh IP
        elif ind["type"] == "ip" and obj_ip == ind["value"]:
            labels.append(ind[target_key])

    # Lấy nhãn cao nhất (mức độ nghiêm trọng nhất)
    return max(labels) if labels else 0

def parse_event(record, scenario):
    datum = get_datum(record)
    evt = datum.get("com.bbn.tc.schema.avro.cdm18.Event")

    # Nếu dòng này không phải Event thì bỏ qua
    if not evt:
        return None

    # Lấy UUID Subject và Object từ Event
    # CDM đôi khi lưu uuid dạng string, đôi khi dạng dict {"uuid": "..."}
    subj_raw = evt.get("subject")
    obj_raw = evt.get("object")

    subj_uuid = subj_raw.get("uuid") if isinstance(subj_raw, dict) else subj_raw
    obj_uuid = obj_raw.get("uuid") if isinstance(obj_raw, dict) else obj_raw

    # --- TRA CỨU THÔNG TIN TỪ PASS 1 (LOOKUP) ---

    # 1. Thông tin Process (Subject)
    s_info = subject_info.get(subj_uuid, {})
    proc_name = s_info.get("proc_name", "unknown")
    principal_uuid = s_info.get("principal_uuid")

    # 2. Thông tin User (Principal)
    u_info = principal_info.get(principal_uuid, {})

    # 3. Thông tin Object (File/IP)
    o_info = object_info.get(obj_uuid, {})
    obj_path = o_info.get("path")
    obj_ip = o_info.get("ip")
    obj_type = o_info.get("type", "UNKNOWN")

    # --- GÁN NHÃN ---
    label_id = match_label(proc_name, obj_path, obj_ip)

    return {
        "timestamp": evt.get("timestampNanos", 0) // 1_000_000_000,
        "operation": evt.get("type"), # Ví dụ: EVENT_READ, EVENT_WRITE

        "proc_uuid": str(subj_uuid),
        "proc_name": proc_name,

        "user_uid": u_info.get("uid"),
        "username": u_info.get("username"),

        "obj_type": obj_type,
        "obj_path": obj_path,
        "obj_ip": obj_ip,

        "label_id": label_id,
        "label_name": LABEL_MAPPING.get(str(label_id), "Benign"),
        "scenario": scenario
    }

# =========================================================
# CHẠY CHƯƠNG TRÌNH
# =========================================================

# Tìm tất cả file JSON (loại bỏ file nén)
files = sorted([p for p in DATA_DIR.glob("**/*.json*") if "tar.gz" not in p.name])

# --- PASS 1: Build Maps (Đọc lướt để lấy Metadata) ---
print(f"🔍 PASS 1: Quét Metadata từ {len(files)} files...")
for f in tqdm(files, desc="Metadata Scan"):
    with open(f, 'r', encoding='utf-8') as fh:
        for line in fh:
            try:
                rec = json.loads(line)
                parse_metadata(rec)
            except:
                continue

print(f"📊 Đã học được: {len(principal_info)} Users, {len(subject_info)} Process, {len(object_info)} Objects")

# --- PASS 2: Parse Events & Write Parquet ---
print("\n🧠 PASS 2: Xử lý Events & Gán nhãn...")
writer = None
buffer = []

for f in files:
    scenario = f.parent.name # Lấy tên thư mục làm tên kịch bản

    with open(f, 'r', encoding='utf-8') as fh:
        for line in tqdm(fh, desc=f.name, leave=False):
            try:
                rec = json.loads(line)
                row = parse_event(rec, scenario)

                if row:
                    buffer.append(row)
            except:
                continue

            # Ghi xuống đĩa khi buffer đầy (tránh tràn RAM)
            if len(buffer) >= BATCH_SIZE:
                table = pa.Table.from_pylist(buffer)
                if writer is None:
                    writer = pq.ParquetWriter(OUTPUT_PARQUET, table.schema, compression="snappy")
                writer.write_table(table)
                buffer.clear()

        gc.collect() # Dọn rác bộ nhớ sau mỗi file

# Ghi nốt phần còn dư
if buffer:
    table = pa.Table.from_pylist(buffer)
    if writer is None:
        writer = pq.ParquetWriter(OUTPUT_PARQUET, table.schema, compression="snappy")
    writer.write_table(table)

if writer:
    writer.close()

print(f"\n✅ HOÀN TẤT! File kết quả: {OUTPUT_PARQUET}")

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install torch-geometric